In [3]:
from moss.mosaic import Mosaic
import nibabel as nib
import multiprocessing
#os and i/o
import os
import numpy as np
import os.path as op
import seaborn as sns
import matplotlib
import scipy
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [4]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects.txt'
subs = list(np.loadtxt(subj_file,'string'))
subs = ['fd_104']
os.chdir(home_dir)

In [105]:
def is_invertible(a):
    return a.shape[0] == a.shape[1] and np.linalg.matrix_rank(a) == a.shape[0]

In [106]:
def compute_inv_shrunk_covariance(x):
    #see http://www.diedrichsenlab.org/pubs/Walther_Neuroimage_2016.pdf
    t,n = x.shape #t measurements by n voxels

    #demean
    x = x - x.mean(0)

    #compute covariance
    sample = (1.0/t) * np.dot(np.transpose(x),x)

    #copute prior
    prior = np.diag(np.diag(sample))

    #compute shrinkage
    d = 1.0/n * np.linalg.norm(sample - prior,ord = 'fro')**2
    y = np.square(x)
    r2 = 1.0/n/t**2 * np.sum(np.sum(np.dot(np.transpose(y),y)))- \
    1.0/n/t*np.sum(np.sum(np.square(sample)))

    #compute the estimator
    shrinkage = max(0,min(1,r2/d))
    sigma = shrinkage*prior + (1-shrinkage)*sample

    #compute the inverse
    if is_invertible(sigma):
        inv_sigma = np.linalg.inv(sigma)
    else:
        inv_sigma = np.linalg.inv(prior) #univariate
    
    return inv_sigma

In [107]:
#make sure MTL masks are exclusive
def trim_mask(mask,m):
    exclusions = overlap_masks[:]
    exclusions.remove(m)

    m1 = op.abspath('./data/' + sub + '/masks/' + exclusions[0] +'.nii.gz')
    m1 = nib.load(m1).get_data().astype(bool)

    m2 = op.abspath('./data/' + sub + '/masks/' + exclusions[1] +'.nii.gz')
    m2 = nib.load(m2).get_data().astype(bool)

    bad = np.logical_or(m1,m1)
    good = np.invert(bad)

    mask = np.logical_and(mask,good)
    return mask

In [110]:
def compute_inverse_sigma(sub,exp,smooth,masks):
    sub_path = op.join(home_dir,'analysis', exp, sub, 'reg','epi', smooth)
    for run in map(str,range(1,4)):
        res = op.join(sub_path, 'run_'  + run,'res4d_xfm.nii.gz')
        res = nib.load(res).get_data().astype(float)

        for m in masks:
            mask = op.join(home_dir,'data', sub,  'masks', m + '.nii.gz')
            mask = nib.load(mask).get_data().astype(bool)
            if m in overlap_masks:
                mask = trim_mask(mask,m)

            x = res[mask]
            x = np.transpose(x)

            inv_sigma = compute_inv_shrunk_covariance(x)

            inv_sigma = scipy.linalg.fractional_matrix_power(inv_sigma,.5) #take square root

            out_f = op.join(home_dir,'covariance','_'.join([exp,sub,run,m]) + '.txt')
            np.savetxt(out_f,inv_sigma)

In [117]:
def delete_inverse_sigma(sub,exp,smooth,masks):
    for run in map(str,range(1,4)):
        for m in masks:
            out_f = op.join(home_dir,'covariance','_'.join([exp,sub,run,m]) + '.txt')
            os.remove(out_f)

In [111]:
def extract_betas(sub,exp,smooth,masks):
    betas = {'sub':[],'mask':[],'run':[],'condition':[],'value':[],'voxel':[],'row':[]}
    for m in masks:
        out_f = op.join(home_dir,'betas', '_'.join([exp,smooth,m]) + '.csv')

        for sub in subs:
            sub_path = op.join(home_dir,'analysis', exp, sub, 'reg','epi', smooth )

            mask = op.join(home_dir,'data', sub,  'masks', m + '.nii.gz')
            mask = nib.load(mask).get_data().astype(bool)
            if m in overlap_masks:
                mask = trim_mask(mask,m)

            for run in map(str,range(1,4)):
                run_dir = op.join(sub_path, 'run_'  + run)

                if os.path.exists(run_dir):

                    for i in range(1,5):
                        f = run_dir + '/cope' + str(i) + '_xfm.nii.gz'
                        cond = cond_map[i]

                        #load stat image
                        stat = nib.load(f).get_data().astype(float)
                        stat = stat[mask]

                        for n,val in enumerate(stat):
                            betas['voxel'].append(n)                        
                            betas['sub'].append(sub)
                            betas['value'].append(val)
                            betas['mask'].append(m)
                            betas['run'].append(run)
                            betas['condition'].append(cond)
                            betas['row'].append(i)
                else:
                    print run_dir

    betas = pd.DataFrame(betas)
    betas = betas.set_index(['sub', 'run','mask','condition'])
    return betas

In [113]:
def prewhiten_betas(sub,betas,masks):
    for run in map(str,range(1,4)):
        for m in masks:
            #load covariance and take root
            inv_sigma = op.join(home_dir,'covariance','_'.join([exp,sub,run,m]) + '.txt')
            inv_sigma = np.loadtxt(inv_sigma)
            
            for cond in cond_map.values():
                vals = betas.loc[(sub,run,m,cond),'value'].values
                whiten_vals = np.dot(vals,inv_sigma)
                betas.loc[(sub,run,m,cond),'value'] = whiten_vals
    return betas

In [115]:
def compute_rsa(sub,exp,masks):
    rsas = {}
    for m in masks:
        
        runs = list(set(map(lambda x: x[0],betas.loc[(sub)].index)))
        nruns = len(runs)
        rsa = np.zeros((nconds,nconds))

        for i in range(1,nconds+1):
            for j in range(1,nconds+1):
                corr = []
                
                corr.append(scipy.stats.pearsonr(betas.loc[(sub,'1',m,cond_map[i]),'value'].values,
                                     betas.loc[(sub,'2',m,cond_map[j]),'value'].values))
                corr.append(scipy.stats.pearsonr(betas.loc[(sub,'2',m,cond_map[i]),'value'].values,
                                     betas.loc[(sub,'1',m,cond_map[j]),'value'].values))

                if nruns == 3:

                    corr.append(scipy.stats.pearsonr(betas.loc[(sub,'1',m,cond_map[i]),'value'].values,
                                         betas.loc[(sub,'3',m,cond_map[j]),'value'].values))
                    corr.append(scipy.stats.pearsonr(betas.loc[(sub,'3',m,cond_map[i]),'value'].values,
                                         betas.loc[(sub,'1',m,cond_map[j]),'value'].values))
                    corr.append(scipy.stats.pearsonr(betas.loc[(sub,'2',m,cond_map[i]),'value'].values,
                                         betas.loc[(sub,'3',m,cond_map[j]),'value'].values))
                    corr.append(scipy.stats.pearsonr(betas.loc[(sub,'3',m,cond_map[i]),'value'].values,
                                         betas.loc[(sub,'2',m,cond_map[j]),'value'].values))

            
                       
                rsa[i-1,j-1]  = np.nanmean(corr)
        rsas[m] = rsa
        
    return rsas


In [121]:
# masks = ['peri_sim','para_sim','hipp','hipp_right','hipp_left',
#          'entorhinal']
masks = ['hipp']
overlap_masks = ['peri_sim','para_sim','hipp']
exp = 'sim_4mm-onebeta'
smooth = 'smoothed'

cond_map = {1:'AB+',2:'AC-',3:'B-',4:'C+'}
nconds = len(cond_map.keys())

subs = ['fd_104']

for sub in subs{}
    compute_inverse_sigma(sub,exp,smooth,masks)
    betas = extract_betas(sub,exp,smooth,masks)

    betas = prewhiten_betas(sub,betas,masks)
    # delete_inverse_sigma(sub,exp,smooth,masks) #save disk space
    rsas = compute_rsa(sub,exp,masks)

    for m in masks:
        all_rsas[m].append(rsas[m])



SyntaxError: invalid syntax (<ipython-input-121-077d14443056>, line 13)

In [120]:
all_rsas = {k: [] for k in masks}
all_rsas

{'entorhinal': [],
 'hipp': [],
 'hipp_left': [],
 'hipp_right': [],
 'para_sim': [],
 'peri_sim': []}

In [90]:
# for sub in subs:
#     for m in ['hipp']:
        
#         #Get crossval folds. Deal with unequal number of runs across subjects
#         runs = list(set(map(lambda x: x[0],betas.loc[(sub)].index)))
#         nruns = len(runs)
#         if nruns == 3:
#             train_runs = [['1','2'],['1','3'],['2','3']]
#         elif nruns == 2:
#             train_runs = [['1'],['2']]
        
#         rsa = np.zeros((4,4,len(runs)))

#         #loop through condition pairs
#         for i in range(1,rsa.shape[0]+1):
#             for j in range(1,rsa.shape[0]+1):
#                 if j < i:
#                     print i,j
#                     #get euclidian distance between conditions for each run
#                     run_euclid_dist = {}
#                     for run in map(str,range(1,nruns+1)):
                        
#                         cond1 = betas.loc[(sub,run,m,cond_map[i]),'value'].values
#                         cond2 = betas.loc[(sub,run,m,cond_map[j]),'value'].values
                        
#                         run_euclid_dist[run] = cond1 - cond2
                    
#                     #do cross-validation
#                     for n,train_run in enumerate(train_runs):
#                         train_vector = []
#                         test_vector = []
                    
#                         for run in map(str,range(1,nruns+1)): 
#                             if run in train_run:
#                                 train_vector.append(run_euclid_dist[run])
#                             else:
#                                 test_vector.append(run_euclid_dist[run])

#                         train_vector = np.mean(train_vector,0)
#                         test_vector = np.mean(test_vector,0)
                        
#                         diff = np.dot(train_vector,test_vector)
                        
#                         rsa[i-1,j-1,n] = diff
# # rsa = rsa.mean(2)

In [101]:
rsa

array([[ 0.03905492,  0.13804446,  0.11054621,  0.11285011],
       [ 0.13804446,  0.17733881,  0.16358399,  0.16732785],
       [ 0.11054621,  0.16358399,  0.23519571,  0.22827077],
       [ 0.11285011,  0.16732785,  0.22827077,  0.26099641]])